# Importing Packages

In [13]:
# importing packages
from itertools import permutations, count
from tqdm import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import set_config
from sklearn.model_selection import (train_test_split,
                                     cross_val_score,
                                     cross_validate,
                                     KFold)
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.compose import TransformedTargetRegressor
import optuna
from sklearn.metrics import *
from sklearn.inspection import *
import shap
import lovelyplots
import statsmodels.api as sm
import gc
import warnings


import scienceplots


import sklearn
from sklearn.model_selection import *
from sklearn.metrics import *
from sklearn.preprocessing import *
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from scipy import stats

import tensorflow as tf
import keras_tuner


import missingno


# Plotting Configuration

In [14]:
plt.rcdefaults()
mpl_global_config = {
    'figure.figsize': (7, 7),
    'figure.dpi': 1000,
    'font.size': 16,
    'axes.labelsize': 14,
    'axes.titlesize': 14,
    'xtick.labelsize': 12,
    'ytick.labelsize': 12,
    'legend.fontsize': 8,
    'lines.linewidth': 2,
    'lines.markersize': 3,
    'grid.linewidth': 0.75,
    'savefig.dpi': 1000,
    'savefig.transparent': False,
    'savefig.bbox': 'tight',
    'pdf.compression': 9,
    'axes.axisbelow': True
}
plt.rcParams.update(mpl_global_config)
plt.style.use(['science', 'nature', 'high-contrast', "no-latex"])


colors = {
    "yellow": "#DDAA33",
    "red": "#BB5566",
    "blue": "#004488",
    "black": "#000000",
    "white": "#FFFFFF"
}

# Global Configuration Setting Controling Randomness, Trials, etc

In [15]:
# config initialization
set_config(transform_output="pandas")
np.seterr(under='ignore')
warnings.filterwarnings('ignore')
SEED = 0
n_trials = 75
%matplotlib inline

# def evaluation

In [16]:
def evaluation(trained_model, X, y_real):

    y_pred = trained_model.predict(X)

    mse = mean_squared_error(y_real, y_pred, squared=True)
    rmse = mean_squared_error(y_real, y_pred, squared=False)
    mae = mean_absolute_error(y_real, y_pred)
    r2 = r2_score(y_real, y_pred)
    maxerror = max_error(y_real, y_pred)
    mape = mean_absolute_percentage_error(y_real, y_pred)

    return {"MeanSquaredError":mse,
            "RootMeanSquaredError": rmse,
            "MeanAbsoluteError": mae,
            "R2": r2,
            "MaxError": maxerror,
            "MAPE": mape}

# Read the Preprocessed Data

In [17]:
# reading the dataset
df = pd.read_csv("df.csv",
                 dayfirst=True,
                 parse_dates=True,
                 index_col="Date")

In [ ]:
df.columns

In [ ]:
df.isnull().sum()

In [20]:
data_summary = df.describe()
data_summary.loc["Missing Values Ratio"] = df.isnull().sum().divide(len(df))
data_summary.loc["Unique Values Ratio"] = df.nunique().divide(len(df))
data_summary.loc["Kurtosis"] = df.kurtosis()
data_summary.loc["Skewness"] = df.skew()
data_summary.T.to_csv("data_summary.csv")

In [ ]:
data_summary.T

# Separating X and y

In [ ]:
X = df.drop(columns=["Total Biogas Flowrate (m3/d)"])
y = df.loc[: , "Total Biogas Flowrate (m3/d)"]
print(f"dataframe shape: {df.shape}\n"\
      f"features shape: {X.shape}\n"\
      f"target shape: {y.shape}")

# Train and Test Split and Normalization

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.30,
    random_state=SEED
)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

# Hyperparameter Optimization

In [ ]:
# train a Decision Tree model with tpe sampler and median pruner
def objective_dt(trial):

    dt_params = {
            "max_depth": trial.suggest_int("max_depth", 2, 15),
            "min_samples_leaf": trial.suggest_float("min_samples_leaf", 0.01, 0.5),
        }

    model = DecisionTreeRegressor(random_state=SEED, **dt_params)

    score = -np.mean(
        cross_val_score(
                model,
                X_train,
                y_train,
                cv=KFold(n_splits=5),
                n_jobs=-1,
                scoring='neg_mean_squared_error')
        )

    return score

study_dt = optuna.create_study(
    direction="minimize",
    sampler=optuna.samplers.TPESampler(seed=SEED),
    pruner=optuna.pruners.HyperbandPruner()
)

study_dt.optimize(objective_dt, n_trials=n_trials)

In [ ]:
# train a Random Forest model with tpe sampler and median pruner
def objective_rf(trial):

    rf_params = {
            "max_features": trial.suggest_float("max_features", 0.3, 1),
            "max_depth": trial.suggest_int("max_depth", 2, 15),
            "min_samples_leaf": trial.suggest_float("min_samples_leaf", 0.01, 0.5),
            "bootstrap": trial.suggest_categorical("bootstrap", [False, True]),
            "n_estimators": trial.suggest_int("n_estimators", 10, 200)
        }

    model = RandomForestRegressor(random_state=SEED, **rf_params)


    score = -np.mean(
        cross_val_score(
                model,
                X_train,
                y_train,
                cv=KFold(n_splits=5),
                n_jobs=-1,
                scoring='neg_mean_squared_error')
        )

    return score

study_rf = optuna.create_study(
    direction="minimize",
    sampler=optuna.samplers.TPESampler(seed=SEED),
    pruner=optuna.pruners.HyperbandPruner()

)

study_rf.optimize(objective_rf, n_trials=n_trials)

In [ ]:
# train an XBGBoost model with tpe sampler and median pruner
def objective_xgb(trial):

    params = {
        "n_estimators": trial.suggest_int("n_estimators", 10, 5000),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 1.0, log=True),
        "max_delta_step": trial.suggest_int("max_delta_step", 0, 20),
        "reg_lambda": trial.suggest_float("reg_lambda",1e-9, 100.0, log=True),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-9, 100.0, log=True),
        "subsample": trial.suggest_float("subsample", 0.1, 1.0, log=True),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.1, 1.0, log=True),
        "max_depth": trial.suggest_int("max_depth", 2, 15),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
        "gamma": trial.suggest_float("gamma", 0.1, 0.5, log=True),
        "scale_pos_weight": trial.suggest_float("scale_pos_weight", 1e-6, 500, log=True)
    }


    model = XGBRegressor(random_state=SEED, objective="reg:squarederror", **params)

    score = -np.mean(
        cross_val_score(
                model,
                X_train,
                y_train,
                cv=KFold(n_splits=5),
                n_jobs=-1,
                scoring='neg_mean_squared_error')
        )

    return score

study_xgb = optuna.create_study(
    direction="minimize",
    sampler=optuna.samplers.TPESampler(seed=SEED),
    pruner=optuna.pruners.HyperbandPruner()
)

study_xgb.optimize(objective_xgb, n_trials=n_trials)

In [55]:
best_hps = {
    "DecisionTree": study_dt.best_params,
    "RandomForest": study_rf.best_params,
    "XGBoost": study_xgb.best_params,
}

In [ ]:
best_hps

In [24]:
best_hps = {'DecisionTree': {'max_depth': 14, 'min_samples_leaf': 0.010039169930176879},
            'RandomForest': {'max_features': 0.38554104875151896,
            'max_depth': 10,
            'min_samples_leaf': 0.010168225931922027,
            'bootstrap': False,
            'n_estimators': 124},
            'XGBoost': {'n_estimators': 4860,
            'learning_rate': 0.49003165411341526,
            'max_delta_step': 20,
            'reg_lambda': 1.1231181925328875e-05,
            'reg_alpha': 3.426318577751964e-08,
            'subsample': 0.89607666469783,
            'colsample_bytree': 0.8147384539995441,
            'max_depth': 4,
            'min_child_weight': 10,
            'gamma': 0.1296891015376799,
            'scale_pos_weight': 2.2740058630279575e-05}}

# Fitting the Models with Best HPs

In [25]:

trained_models = {
    "DecisionTree": DecisionTreeRegressor(random_state=SEED,
                                          **best_hps["DecisionTree"]).fit(X_train, y_train),

    "RandomForest": RandomForestRegressor(random_state=SEED,
                                          **best_hps["RandomForest"]).fit(X_train, y_train),


    "XGBoost": XGBRegressor(random_state=SEED,
                            **best_hps["XGBoost"],
                            objective="reg:squarederror").fit(X_train, y_train),

}


In [26]:
pd.DataFrame.from_dict(best_hps, orient="index").T.to_csv("hyperparameters_when_all_features_are_chosen.csv")

# Evaluation

In [27]:
evaluations_dictionary = {
    "DecisionTree_Train": evaluation(trained_models["DecisionTree"],
                                     X_train, y_train),
    "DecisionTree_Test": evaluation(trained_models["DecisionTree"],
                                    X_test, y_test),

    "RandomForest_Train": evaluation(trained_models["RandomForest"],
                                     X_train, y_train),
    "RandomForest_Test": evaluation(trained_models["RandomForest"],
                                    X_test, y_test),

    "XGBoost_Train": evaluation(trained_models["XGBoost"],
                                X_train, y_train),
    "XGBoost_Test": evaluation(trained_models["XGBoost"],
                               X_test, y_test),

}

evaluation_df = pd.DataFrame(evaluations_dictionary)
evaluation_df.T.to_csv("model_evaluation.csv")


In [ ]:
evaluation_df.T

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Define the R² values for each model
models = ['ANN', 'Decision Tree', 'Random Forest', 'XGBoost']

# R² values for training and testing data
train_r2 = [0.81, 0.92, 0.95, 1.00]  # R² for Train
test_r2 = [0.80, 0.76, 0.86, 0.97]   # R² for Test

# Function to create radar plot
def create_combined_radar_plot(train_data, test_data):
    num_vars = len(models)

    # Compute angle for each axis
    angles = np.linspace(0, 2 * np.pi, num_vars, endpoint=False).tolist()

    # Complete the loop by appending the first value to the end
    train_data = np.concatenate((train_data, [train_data[0]]))
    test_data = np.concatenate((test_data, [test_data[0]]))
    angles += angles[:1]

    # Create the radar chart
    fig, ax = plt.subplots(figsize=(8, 8), subplot_kw=dict(polar=True))
    
    # Plot training data
    ax.fill(angles, train_data, color='blue', alpha=0.25, label='Training R²')
    ax.plot(angles, train_data, color='blue', linewidth=2)
    
    # Plot testing data
    ax.fill(angles, test_data, color='orange', alpha=0.25, label='Testing R²')
    ax.plot(angles, test_data, color='orange', linewidth=2)

    # Add labels to each axis
    plt.xticks(angles[:-1], models)
    
    # Add title and legend
    plt.title('R² Values for Training and Testing Metrics')
    plt.legend(loc='upper right')

# Create combined radar plot for R² values
create_combined_radar_plot(train_r2, test_r2)
plt.show()


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Create a DataFrame with the provided data
data = {
    'Model': ['ANN', 'ANN', 'DT', 'DT', 'RF', 'RF', 'XGBoost', 'XGBoost'],
    'Phase': ['Train', 'Test', 'Train', 'Test', 'Train', 'Test', 'Train', 'Test'],
    'RMSE': [4297, 4332, 2684, 4914, 2089, 3779, 9, 1809],
    'MAE': [2664, 2562, 1305, 2412, 1070, 1969, 5, 1148],
    'R2': [0.81, 0.80, 0.92, 0.76, 0.95, 0.86, 1.00, 0.97],
    'MAPE': [0.09, 0.08, 0.04, 0.08, 0.04, 0.06, 0.00, 0.04]
}

df = pd.DataFrame(data)

# Set the width of the bars
bar_width = 0.35

# Set positions of bar on X axis
r1 = np.arange(len(df[df['Phase'] == 'Train']))
r2 = [x + bar_width for x in r1]

# Create subplots for each metric
metrics = ['RMSE', 'MAE', 'R2', 'MAPE']
fig, axs = plt.subplots(2, 2, figsize=(12, 10))
axs = axs.flatten()

# Loop through each metric and create a bar plot
for i, metric in enumerate(metrics):
    # Plotting training data
    train_bars = axs[i].bar(r1, df[df['Phase'] == 'Train'][metric], color='blue', width=bar_width / 2,
                            label='Train')
    
    # Plotting testing data
    test_bars = axs[i].bar(r2, df[df['Phase'] == 'Test'][metric], color='orange', width=bar_width / 2,
                           label='Test')

    # Adding labels and title
    axs[i].set_xlabel('Models')
    axs[i].set_ylabel(metric)
    axs[i].set_title(f'{metric} Comparison')
    axs[i].set_xticks([r + bar_width / 4 for r in r1])
    axs[i].set_xticklabels(df['Model'].unique())
    axs[i].legend()

    # Adding values on top of bars
    for bar in train_bars:
        yval = bar.get_height()
        axs[i].text(bar.get_x() + bar.get_width()/2 - bar_width/4, yval + (yval * 0.02), 
                     f'{yval:.2f}', ha='center', va='bottom')

    for bar in test_bars:
        yval = bar.get_height()
        axs[i].text(bar.get_x() + bar.get_width()/2 + bar_width/4, yval + (yval * 0.02), 
                     f'{yval:.2f}', ha='center', va='bottom')
    plt.savefig(f"evaluation metrics comparision.svg")
# Adjust layout
plt.tight_layout()
plt.show()




In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Define the R² values for each model
models = ['ANN', 'Decision Tree', 'Random Forest', 'XGBoost']

# R² values for training and testing data
train_r2 = [0.81, 0.92, 0.95, 1.00]  # R² for Train
test_r2 = [0.80, 0.76, 0.86, 0.97]   # R² for Test

# Function to create an improved radar plot
def create_improved_radar_plot(train_data, test_data):
    num_vars = len(models)

    # Compute angle for each axis
    angles = np.linspace(0, 2 * np.pi, num_vars, endpoint=False).tolist()

    # Complete the loop by appending the first value to the end
    train_data = np.concatenate((train_data, [train_data[0]]))
    test_data = np.concatenate((test_data, [test_data[0]]))
    angles += angles[:1]

    # Create the radar chart with higher resolution
    fig, ax = plt.subplots(figsize=(8, 8), subplot_kw=dict(polar=True), dpi=150)
    
    # Plot training data with bold lines and black color
    ax.fill(angles, train_data, color='blue', alpha=0.25, label='Training R²')
    ax.plot(angles, train_data, color='blue', linewidth=3)  # Bold line
    
    # Plot testing data with bold lines and a different color
    ax.fill(angles, test_data, color='red', alpha=0.25, label='Testing R²')
    ax.plot(angles, test_data, color='red', linewidth=3)  # Bold line

    # Add labels to each axis in bold font
    plt.xticks(angles[:-1], models, fontweight='bold')
    
    # Add title and legend with bold font
    plt.title('R² Values for Training and Testing Metrics', fontweight='bold')
    plt.legend(loc='upper right', frameon=True)

# Create improved radar plot for R² values
create_improved_radar_plot(train_r2, test_r2)
plt.show()


In [29]:
plt.style.use(['science', 'nature', 'high-contrast', "no-latex"])


In [ ]:
for model in trained_models.keys():
    for dataset in [("Train", X_train, y_train), ("Test", X_test, y_test)]:
        data_type, X_data, y_data = dataset
        fig, ax = plt.subplots(figsize=(7, 7))
        PredictionErrorDisplay.from_estimator(
            trained_models[model],
            X_data,
            y_data,
            kind="actual_vs_predicted",
            scatter_kwargs={"alpha": 1,
                            "color": "#004488",  
                            "label": f"{data_type} MAE: {int(evaluation_df.loc['MeanAbsoluteError', f'{model}_{data_type}'].round(0))}", 
                            "s":45},
            line_kwargs={"color": "black"},
            ax=ax)

        ax.set_ylabel(f"Actual {y.name}", fontsize=14, fontweight='bold')  
        ax.set_xlabel(f"Predicted {y.name}", fontsize=14, fontweight='bold')  
        ax.legend(fontsize=9, loc="upper left", fancybox=True, shadow=True, ncol=1, frameon=False)  
        ax.tick_params(axis='x', labelsize=12)  # Increased font size for x-axis numbers
        ax.tick_params(axis='y', labelsize=12)  # Increased font size for y-axis numbers
        
        from matplotlib.ticker import AutoMinorLocator  # Import minor locator
        ax.xaxis.set_minor_locator(AutoMinorLocator(2))  # Add minor ticks to x-axis
        ax.yaxis.set_minor_locator(AutoMinorLocator(2))  # Add minor ticks to y-axis
        
        ax.grid(False)  # Disable gridlines
        plt.tight_layout()
        plt.savefig(f"evaluation actual_vs_predicted_{model}_{data_type}.svg")
        # plt.close()
        # gc.collect()

In [ ]:
for model in trained_models.keys():
    fig, ax = plt.subplots(figsize=(7, 7))
    PredictionErrorDisplay.from_estimator(
        trained_models[model],
        X_test,
        y_test,
        kind="residual_vs_predicted",
        scatter_kwargs={"alpha": 1,
                        "s":40,
                        "color": "#DD3C2D",
                        "label": f"Test R2: {evaluation_df.loc['R2', f'{model}_Test'].round(3)}",
                        },
        line_kwargs={"color": "black"},
        ax=ax)


    PredictionErrorDisplay.from_estimator(
        trained_models[model],
        X_train,
        y_train,
        kind="residual_vs_predicted",
        scatter_kwargs={"alpha": 0.3,
                        "s":40,
                        "color": "#364B9A",
                        "label": f"Train R2: {evaluation_df.loc['R2', f'{model}_Train'].round(3)}",
                        "marker": "X"
                        },
        line_kwargs={"color": "black"},
        ax=ax)



    ax.set_xlabel(f"Predicted {y.name}", fontsize=14)
    ax.set_ylabel(f"Residuals\n(Actual - Predicted)", fontsize=14)
    ax.legend(fontsize=9, loc="lower left", fancybox=True, shadow=True, ncol=1)
    ax.tick_params(axis='x', labelsize=9)
    ax.tick_params(axis='y', labelsize=9)
    plt.tight_layout()
    plt.savefig(f"evaluation_residual_{model}.svg", format='svg', dpi=1000, bbox_inches='tight')
    # plt.close()
    # gc.collect()

In [ ]:
for model in trained_models.keys():
    residuals = y_test - trained_models[model].predict(X_test)
    fig, ax = plt.subplots(figsize=(7, 7))
    sm.qqplot(residuals, line ='q', ax=ax)
    ax.set_ylabel("Sample Quantiles", fontsize=14)
    ax.set_xlabel("Theoretical Quantiles", fontsize=14)
    ax.tick_params(axis='x', labelsize=9)
    ax.tick_params(axis='y', labelsize=9)
    plt.savefig(f"evaluation qqplot_{model}.svg", format='svg', dpi=1000, bbox_inches='tight')
    # plt.close()
    # gc.collect()

In [ ]:
for model in trained_models.keys():
    residuals_test = y_test - trained_models[model].predict(X_test)
    fig, ax = plt.subplots(figsize=(7, 7))
    ax.text(0.63, 0.7,
            f"Mean: {residuals_test.mean().round(3)}\nStd: {residuals_test.std().round(3)}",
            transform=ax.transAxes, fontsize=9)
    sns.kdeplot(residuals_test, ax=ax, label=f"{model}", color="#6EA5CD", linewidth=2)
    ax.set_ylabel("Density", fontsize=14)
    ax.set_xlabel("Error", fontsize=14)
    plt.legend(fontsize=9, loc="upper left", fancybox=True, shadow=True, ncols=1)
    ax.tick_params(axis='x', labelsize=9)
    ax.tick_params(axis='y', labelsize=9)
    plt.savefig(f"Evaluation_kde_residuals_{model}.svg", format='svg', dpi=1000, bbox_inches='tight')
#     plt.close()
#     gc.collect()

## Explanation

## SHAP

In [34]:
plt.rcdefaults()
# plt.style.use()


mpl_global_config = {
    'figure.dpi': 100,
    'savefig.dpi': 1200,
    'savefig.transparent': False,
    'savefig.bbox': 'tight',
    'axes.axisbelow': True,
    'axes.spines.top': True,
    'axes.spines.right': True,
    'axes.spines.bottom': True,
    'axes.spines.left': True,
    "axes.grid": True,
    'grid.linewidth': 0.3,

}


plt.rcParams.update(mpl_global_config)

In [35]:
explainer = shap.TreeExplainer(trained_models["XGBoost"])
shap_values = explainer.shap_values(X_test)
shap_interaction_values = explainer.shap_interaction_values(X_test)

In [ ]:
shap.summary_plot(shap_values, X_test, plot_type="bar", color="#DD3C2D", show=False, max_display=30)
plt.gca().spines['top'].set_visible(True)
plt.gca().spines['right'].set_visible(True)
plt.gca().spines['bottom'].set_visible(True)
plt.gca().spines['left'].set_visible(True)
plt.gca().spines['top'].set_color('black')
plt.gca().spines['right'].set_color('black')
plt.gca().spines['bottom'].set_color('black')
plt.gca().spines['left'].set_color('black')
plt.tight_layout()
plt.savefig("Explanation_FeatureImportancePlot.svg", format='svg', dpi=1000, bbox_inches='tight')
# plt.close()
# gc.collect()

In [ ]:
import matplotlib.pyplot as plt
import shap

# Generate the SHAP summary plot
shap.summary_plot(shap_values, X_test, show=False, max_display=30)

# Customize spines (axis lines)
ax = plt.gca()  # Get current axes
ax.spines['top'].set_visible(True)
ax.spines['right'].set_visible(True)
ax.spines['bottom'].set_visible(True)
ax.spines['left'].set_visible(True)

# Set spine colors and widths
for spine in ax.spines.values():
    spine.set_color('black')  # Set spine color to black
    spine.set_linewidth(2)     # Make spines bolder

# Set title and labels in bold black
plt.title('SHAP Summary Plot', color='black', fontsize=14, fontweight='bold')  # Title in bold black

# Customize tick parameters to bold the numbers
ax.tick_params(axis='both', which='major', labelcolor='black', labelsize=12)  # Increased font size for major ticks
for tick in ax.get_xticklabels() + ax.get_yticklabels():  # Get all tick labels
    tick.set_fontsize(12)  # Increased font size for tick labels
    tick.set_fontweight('bold')  # Set font weight to bold

# Increase space between x label and axis
ax.set_xlabel('SHAP Value (impact on model output)', labelpad=15, color='black', fontsize=12, fontweight='bold')  # Adjust labelpad as needed

# Save the plot with a specified filename
plt.savefig("Explanation_SummaryImportancePlot_2.svg", format='svg', dpi=3000, bbox_inches='tight')

In [ ]:
import matplotlib.pyplot as plt
import shap

for main_feature in list(X_test.columns):
    fig, ax = plt.subplots(figsize=(7, 7))  # Set figure size
    
    # Generate the SHAP dependence plot
    shap.dependence_plot(
        (main_feature, main_feature),
        shap_interaction_values, X_test,
        display_features=X_test.iloc[:, :],
        dot_size=50,  # Increase dot size for better visibility
        show=False,
        ax=ax
    )

    # Customize spines (axis lines)
    ax.spines['top'].set_visible(True)
    ax.spines['right'].set_visible(True)
    ax.spines['bottom'].set_visible(True)
    ax.spines['left'].set_visible(True)
    
    # Set spine colors and widths
    for spine in ax.spines.values():
        spine.set_color('black')  # Set spine color to black
        spine.set_linewidth(2)     # Make spines bolder

    # Set title and labels in bold black
    plt.title(f'SHAP Dependence Plot for {main_feature}', color='black', fontsize=14, fontweight='bold')  # Title in bold black
    plt.xlabel(main_feature, color='black', fontsize=12, fontweight='bold')  # X-axis label in bold black
    plt.ylabel('SHAP Value', color='black', fontsize=12, fontweight='bold')  # Y-axis label in bold black

    # Customize tick parameters to bold the numbers
    ax.tick_params(axis='both', which='major', labelcolor='black', labelsize=10)  # Major ticks in black
    for tick in ax.get_xticklabels() + ax.get_yticklabels():  # Get all tick labels
        tick.set_fontsize(10)  # Set font size for tick labels
        tick.set_fontweight('bold')  # Set font weight to bold

    # Manually plot the markers in blue
    scatter = ax.collections[0]  # Get the scatter collection from the dependence plot
    scatter.set_facecolor('blue')  # Change marker color to blue

    # Save the plot with a modified filename to avoid issues with slashes
    main_feature_safe = main_feature.replace("/", "")  # Remove slashes from feature names
    plt.savefig(f"Explanation_Singular_{main_feature_safe}_dependence_plot.svg", format='svg', dpi=3000, bbox_inches='tight')



In [ ]:
# compute SHAP values
shap_values = explainer(X_test[:1000])

shap.plots.heatmap(shap_values, instance_order=shap_values.sum(1))



In [ ]:
df.columns

In [ ]:
shap.plots.scatter(shap_values[:, "Temperature (Degrees Celsius)"], color=shap_values[:, "Alkalinity (mg CaCO3/L)"])


In [42]:
clustering = shap.utils.hclust(X, y)


In [ ]:
shap.plots.bar(shap_values, clustering=clustering, clustering_cutoff=0.8, show=False)

# Customize the ticks and labels for the existing plot
plt.xticks(fontsize=18, fontweight="bold", color="black")  # Bold and black x-ticks
plt.yticks(fontsize=18, fontweight="bold", color="black")  # Bold and black y-ticks

# Customize the axis labels
plt.xlabel(plt.gca().get_xlabel(), fontsize=14, fontweight="bold", color="black")  # Bold and black x-axis label
plt.ylabel(plt.gca().get_ylabel(), fontsize=14, fontweight="bold", color="black")  # Bold and black y-axis label


plt.savefig("bar_plot.svg", format='svg', dpi=3000, bbox_inches='tight')

# Display the updated plot
plt.show()


In [ ]:
shap.plots.bar(shap_values, clustering=clustering, clustering_cutoff=0.7)


In [ ]:
shap.plots.beeswarm(shap_values)


In [ ]:
import matplotlib.pyplot as plt
import shap

# Create the SHAP scatter plot
scatter_plot = shap.plots.scatter(shap_values[:, "Temperature (Degrees Celsius)"], show=False)

# Customize the ticks and labels for the existing plot
plt.xticks(fontsize=12, fontweight="bold", color="black")  # Bold and black x-ticks
plt.yticks(fontsize=12, fontweight="bold", color="black")  # Bold and black y-ticks

# Customize the axis labels
plt.xlabel(plt.gca().get_xlabel(), fontsize=14, fontweight="bold", color="black")  # Bold and black x-axis label
plt.ylabel(plt.gca().get_ylabel(), fontsize=14, fontweight="bold", color="black")  # Bold and black y-axis label

# Display the updated plot
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import shap

# Generate SHAP scatter plots for each feature in X_test
for main_feature in X_test.columns:
    fig, ax = plt.subplots(figsize=(7, 7))  # Set figure size
    
    # Generate the SHAP scatter plot for the current feature
    shap.plots.scatter(shap_values[:, main_feature], show=False, ax=ax)

    # Customize spines (axis lines)
    ax.spines['top'].set_visible(True)
    ax.spines['right'].set_visible(True)
    ax.spines['bottom'].set_visible(True)
    ax.spines['left'].set_visible(True)

    # Set spine colors and widths
    for spine in ax.spines.values():
        spine.set_color('black')  # Set spine color to black
        spine.set_linewidth(2)     # Make spines bolder

    # Set title and labels in bold black
    plt.title(f'SHAP Scatter Plot for {main_feature}', color='black', fontsize=14, fontweight='bold')
    plt.xlabel(main_feature, color='black', fontsize=12, fontweight='bold')
    plt.ylabel('SHAP Value', color='black', fontsize=12, fontweight='bold')

    # Customize tick parameters to bold the numbers
    ax.tick_params(axis='both', which='major', labelcolor='black', labelsize=10)
    for tick in ax.get_xticklabels() + ax.get_yticklabels():
        tick.set_fontsize(10)
        tick.set_fontweight('bold')

    # Manually plot the markers in blue
    scatter = ax.collections[0]  # Get the scatter collection from the scatter plot
    scatter.set_facecolor('blue')  # Change marker color to blue

    # Save the plot with a modified filename to avoid issues with slashes
    main_feature_safe = main_feature.replace("/", "")  # Remove slashes from feature names
    plt.savefig(f"Explanation_Singular_{main_feature_safe}_scatter_plot.svg", format='svg', dpi=3000, bbox_inches='tight')

# Optionally show the last plot (if needed)
plt.show()

In [ ]:

# Create a SHAP force plot
shap.force_plot(
    explainer.expected_value, 
    shap_values.values, 
    X_test, 
    feature_names=['Temperature (Degrees Celsius)', 'DS of Influent Primary Sludge (%)',
                   'Influent Primary to Waste Sludge flowrate Ratio',"Alkalinity (mg CaCO3/L)"],
)



In [ ]:
import numpy as np

# Sum of absolute SHAP values for each instance
impact_scores = np.sum(np.abs(shap_values.values), axis=1)

# Find the instance with the highest impact
highest_impact_index = np.argmax(impact_scores)
highest_impact_instance = shap_values[highest_impact_index]

print(f"Highest impact instance index: {highest_impact_index}")


In [ ]:
# Initialize JavaScript visualization for SHAP
shap.initjs()

# Force plot for one instance
shap.force_plot(
    explainer.expected_value,  # Model's base value
    shap_values[highest_impact_index].values,     # SHAP values for the instance
    X_test.iloc[highest_impact_index]             # Feature values for the instance
)

In [ ]:
# Select the best instance (index or specific row)
instance = X_test.iloc[highest_impact_index]

# Generate the waterfall plot
shap.waterfall_plot(
    shap.Explanation(values=shap_values[highest_impact_index].values,
                     base_values=shap_values[highest_impact_index].base_values,
                     data=instance)
)

In [ ]:
import numpy as np

# Sum of absolute SHAP values for each instance
impact_scores = np.sum(np.abs(shap_values.values), axis=1)

# Find the instance with the highest impact
lowest_impact_index = np.argmin(impact_scores)
lowest_impact_instance = shap_values[highest_impact_index]

print(f"Lowest impact instance index: {lowest_impact_index}")


In [ ]:
# Initialize JavaScript visualization for SHAP
shap.initjs()

# Force plot for one instance
shap.force_plot(
    explainer.expected_value,  # Model's base value
    shap_values[lowest_impact_index].values,     # SHAP values for the instance
    X_test.iloc[lowest_impact_index]             # Feature values for the instance
)

In [ ]:
# Select the best instance (index or specific row)
instance = X_test.iloc[lowest_impact_index]

# Generate the waterfall plot
shap.waterfall_plot(
    shap.Explanation(values=shap_values[lowest_impact_index].values,
                     base_values=shap_values[lowest_impact_index].base_values,
                     data=instance)
)

In [ ]:
# Calculate the variability of SHAP values (e.g., standard deviation) for each instance
shap_variability = np.std(shap_values.values, axis=1)

# Find the instance with the highest variability
best_instance_index = np.argmax(shap_variability)
best_instance = X_test.iloc[best_instance_index]


# Select the best instance (index or specific row)
instance = X_test.iloc[best_instance_index]

# Generate the waterfall plot
shap.waterfall_plot(
    shap.Explanation(values=shap_values[best_instance_index].values,
                     base_values=shap_values[best_instance_index].base_values,
                     data=instance)
)


In [ ]:
# Decision plot for multiple instances
shap.decision_plot(explainer.expected_value, shap_values.values[:100], X_test.iloc[:100])


In [ ]:

# Generate SHAP scatter plots for each feature in X_test
for main_feature in X_test.columns:
    for color_feature in X_test.columns:
        if main_feature != color_feature:  # Avoid coloring by the same feature
            fig, ax = plt.subplots(figsize=(7, 7))  # Set figure size
            
            # Generate the SHAP scatter plot for the current feature
            shap.plots.scatter(
                shap_values[:, main_feature],  # SHAP values for the current feature
                color=shap_values[:, color_feature],  # Color based on another feature's SHAP values
                show=False, 
                ax=ax
            )

            # Customize spines (axis lines)
            for spine in ax.spines.values():
                spine.set_visible(True)
                spine.set_color('black')  # Set spine color to black
                spine.set_linewidth(2)     # Make spines bolder

            # Set title and labels in bold black
            plt.title(f'SHAP Scatter Plot for {main_feature} colored by {color_feature}', 
                      color='black', fontsize=14, fontweight='bold')
            plt.xlabel(main_feature, color='black', fontsize=12, fontweight='bold')
            plt.ylabel('SHAP Value', color='black', fontsize=12, fontweight='bold')

            # Customize tick parameters to bold the numbers
            ax.tick_params(axis='both', which='major', labelcolor='black', labelsize=10)
            for tick in ax.get_xticklabels() + ax.get_yticklabels():
                tick.set_fontsize(10)
                tick.set_fontweight('bold')

            # Manually plot the markers in blue
            scatter = ax.collections[0]  # Get the scatter collection from the scatter plot
            scatter.set_facecolor('blue')  # Change marker color to blue

            # Save the plot with a modified filename to avoid issues with slashes
            main_feature_safe = main_feature.replace("/", "")  # Remove slashes from feature names
            color_feature_safe = color_feature.replace("/", "")  # Remove slashes from feature names
            plt.savefig(f"Explanation_Singular_{main_feature_safe}_colored_by_{color_feature_safe}_scatter_plot.svg", 
                        format='svg', dpi=3000, bbox_inches='tight')

# Optionally show the last plot (if needed)
plt.show()


In [48]:
# download results
import os
from zipfile import ZipFile

with ZipFile("TreeBasedModels.zip", "w") as z:
    for file in os.listdir():
        if file.endswith((".svg", ".csv")):
            z.write(file)